In [3]:
import pandas as pd 
# Load the Excel file
df = pd.read_excel("Online Retail.xlsx")

# Check the first few rows of the DataFrame
print(df.head())

  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

          InvoiceDate  UnitPrice  CustomerID         Country  
0 2010-12-01 08:26:00       2.55     17850.0  United Kingdom  
1 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
2 2010-12-01 08:26:00       2.75     17850.0  United Kingdom  
3 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
4 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  


In [4]:
print(df.dtypes)


InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
Country                object
dtype: object


In [37]:
# Afficher les informations générales
print("--- Informations générales ---")
print(df.info())

# Afficher le nombre de valeurs manquantes
print("\n--- Valeurs manquantes ---")
print(df.isnull().sum())

# Afficher les premières lignes du DataFrame
print("\n--- Aperçu des données ---")
print(df.head())

--- Informations générales ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB
None

--- Valeurs manquantes ---
InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

--- Aperçu des données ---
  InvoiceNo StockCode      

In [38]:
missing_before = df.isnull().sum() 

df.dropna(subset=['CustomerID', 'InvoiceNo', 'StockCode'], inplace=True) 
df['Description'].fillna('Inconnu', inplace=True)  # Exemple pour une variable catégorielle
df['Quantity'].fillna(df['Quantity'].median(), inplace=True)  # Exemple pour une variable numérique
df['UnitPrice'].fillna(df['UnitPrice'].mean(), inplace=True)  # Remplir avec la moyenne pour les prix
df['Country'].fillna(df['Country'].mode()[0], inplace=True)  # Remplir avec la modalité la plus fréquente
after_missing = df.isnull().sum()


 before  InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64
 after  InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64


C:\Users\YacineZERIKAT\AppData\Local\Temp\ipykernel_4648\2481476801.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Description'].fillna('Inconnu', inplace=True)  # Exemple pour une variable catégorielle
C:\Users\YacineZERIKAT\AppData\Local\Temp\ipykernel_4648\2481476801.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which

In [39]:
cancelled_count = df['InvoiceNo'].str.startswith('C').sum()

print(f"Nombre de factures annulées : {cancelled_count}")

Nombre de factures annulées : 8905


In [42]:
df = df[~df['InvoiceNo'].str.startswith('C')]
cancelled_count = df['InvoiceNo'].str.startswith('C').sum()

print(f"Nombre de factures annulées : {cancelled_count}")

TypeError: bad operand type for unary ~: 'float'

In [ ]:
def aggregate_world_data(df, mapping_df):
    # Fusionner avec le mapping des continents
    df = df.merge(mapping_df, left_on='Country', right_on='Country', how='left')

    # Ajouter la colonne des revenus
    df['TotalRevenue'] = df['Quantity'] * df['UnitPrice']

    # Classer les continents selon les dépenses
    continent_revenue = (
        df.groupby('Continent')['TotalRevenue']
        .sum()
        .reset_index()
        .sort_values(by='TotalRevenue', ascending=False)
    )

    # Trouver le continent avec le plus d'opérations annulées
    canceled_df = df[df['Quantity'] < 0]
    canceled_operations = (
        canceled_df.groupby('Continent')['InvoiceNo']
        .count()
        .reset_index()
        .sort_values(by='InvoiceNo', ascending=False)
    )

    # Afficher les résultats
    print("Classement des continents selon les dépenses :")
    print(continent_revenue)
    print("\nContinent avec le plus d'opérations annulées :")
    print(canceled_operations)

In [11]:
country_continent_mapping = {
    "United Kingdom": "Europe", "France": "Europe", "Australia": "Oceania",
    "Netherlands": "Europe", "Germany": "Europe", "Norway": "Europe",
    "EIRE": "Europe", "Switzerland": "Europe", "Spain": "Europe",
    "Poland": "Europe", "Portugal": "Europe", "Italy": "Europe",
    "Belgium": "Europe", "Lithuania": "Europe", "Japan": "Asia",
    "Iceland": "Europe", "Channel Islands": "Europe", "Denmark": "Europe",
    "Cyprus": "Europe", "Sweden": "Europe", "Finland": "Europe",
    "Austria": "Europe", "Greece": "Europe", "Singapore": "Asia",
    "Lebanon": "Asia", "United Arab Emirates": "Asia", "Israel": "Asia",
    "Saudi Arabia": "Asia", "Czech Republic": "Europe", "Canada": "North America",
    "Unspecified": "Unknown", "Brazil": "South America", "USA": "North America",
    "European Community": "Europe", "Bahrain": "Asia", "Malta": "Europe", "RSA": "Africa"
}

# Créer un DataFrame à partir du dictionnaire
mapping_df = pd.DataFrame(list(country_continent_mapping.items()), columns=['Country', 'Continent'])

# Afficher le DataFrame
print(mapping_df)


                 Country      Continent
0         United Kingdom         Europe
1                 France         Europe
2              Australia        Oceania
3            Netherlands         Europe
4                Germany         Europe
5                 Norway         Europe
6                   EIRE         Europe
7            Switzerland         Europe
8                  Spain         Europe
9                 Poland         Europe
10              Portugal         Europe
11                 Italy         Europe
12               Belgium         Europe
13             Lithuania         Europe
14                 Japan           Asia
15               Iceland         Europe
16       Channel Islands         Europe
17               Denmark         Europe
18                Cyprus         Europe
19                Sweden         Europe
20               Finland         Europe
21               Austria         Europe
22                Greece         Europe
23             Singapore           Asia


In [10]:
def aggregate_world_data(df ):
       country_continent_mapping = {
    "United Kingdom": "Europe", "France": "Europe", "Australia": "Oceania",
    "Netherlands": "Europe", "Germany": "Europe", "Norway": "Europe",
    "EIRE": "Europe", "Switzerland": "Europe", "Spain": "Europe",
    "Poland": "Europe", "Portugal": "Europe", "Italy": "Europe",
    "Belgium": "Europe", "Lithuania": "Europe", "Japan": "Asia",
    "Iceland": "Europe", "Channel Islands": "Europe", "Denmark": "Europe",
    "Cyprus": "Europe", "Sweden": "Europe", "Finland": "Europe",
    "Austria": "Europe", "Greece": "Europe", "Singapore": "Asia",
    "Lebanon": "Asia", "United Arab Emirates": "Asia", "Israel": "Asia",
    "Saudi Arabia": "Asia", "Czech Republic": "Europe", "Canada": "North America",
    "Unspecified": "Unknown", "Brazil": "South America", "USA": "North America",
    "European Community": "Europe", "Bahrain": "Asia", "Malta": "Europe", "RSA": "Africa"
}
       
       mapping_df = pd.DataFrame(list(country_continent_mapping.items()), columns=['Country', 'Continent'])


    # Fusionner le DataFrame des transactions avec le mapping des continents
       df = df.merge(mapping_df, left_on='Country', right_on='Country', how='left')

    # Calculer le montant total par transaction
       df['TotalAmount'] = df['Quantity'] * df['UnitPrice']
       valid_sales = df[df['TotalAmount'] > 0]

    # Classer les continents selon les dépenses totales
       continent_sales = valid_sales.groupby('Continent')['TotalAmount'].sum().reset_index()
       continent_sales = continent_sales.sort_values(by='TotalAmount', ascending=False)

    # Identifier le continent avec le plus d’opérations annulées (quantité négative)
       cancelled_transactions = df[df['Quantity'] < 0]
       continent_cancellations = cancelled_transactions['Continent'].value_counts().reset_index()
       continent_cancellations.columns = ['Continent', 'CancelledTransactions']

    # Résultats
       print("Classement des continents par dépenses :")
       print(continent_sales)

       print("\nContinent avec le plus d'opérations annulées :")
       print(continent_cancellations)

       return continent_sales, continent_cancellations









In [11]:
aggregate_world_data(df)

Classement des continents par dépenses :
       Continent   TotalAmount
2         Europe  1.042700e+07
4        Oceania  1.385213e+05
1           Asia  7.132714e+04
3  North America  7.246770e+03
6        Unknown  4.749790e+03
5  South America  1.143600e+03
0         Africa  1.002310e+03

Continent avec le plus d'opérations annulées :
       Continent  CancelledTransactions
0         Europe                  10386
1  North America                    112
2        Oceania                     74
3           Asia                     48


(       Continent   TotalAmount
 2         Europe  1.042700e+07
 4        Oceania  1.385213e+05
 1           Asia  7.132714e+04
 3  North America  7.246770e+03
 6        Unknown  4.749790e+03
 5  South America  1.143600e+03
 0         Africa  1.002310e+03,
        Continent  CancelledTransactions
 0         Europe                  10386
 1  North America                    112
 2        Oceania                     74
 3           Asia                     48)

In [27]:

def aggregate_supplier_data(df):
        supplier_df=pd.read_csv('Supplier.csv')
        
    # Fusionner les dataframes sur 'InvoiceNo'
        merged_df = pd.merge(df, supplier_df, on='InvoiceNo')

    # Filtrer les transactions annulées
        merged_df = merged_df[merged_df['Quantity'] > 0]
    
    # Calculer le total des ventes par fournisseur
        supplier_sales = (
        merged_df.groupby('Fournisseur')['Quantity']
        .sum()
        .reset_index()
        .sort_values(by='Quantity', ascending=False)
    )

    # Filtrer pour l'année 2011 et le Royaume-Uni
        uk_2011_df = merged_df[(merged_df['InvoiceDate'].dt.year == 2011) & (merged_df['Country'] == 'United Kingdom')]
        uk_2011_supplier_sales = (
        uk_2011_df.groupby('Fournisseur')['Quantity']
        .sum()
        .reset_index()
        .sort_values(by='Quantity', ascending=False)
    )

    # Afficher les résultats
        print("Classement des fournisseurs selon le total des ventes :")
        print(supplier_sales)
        print("\nClassement des fournisseurs en 2011 pour le Royaume-Uni :")
        print(uk_2011_supplier_sales)


In [28]:
aggregate_supplier_data(df)

Classement des fournisseurs selon le total des ventes :
  Fournisseur  Quantity
0        F161         1
1        F516         1
2        F703         1

Classement des fournisseurs en 2011 pour le Royaume-Uni :
  Fournisseur  Quantity
0        F161         1
1        F516         1
2        F703         1
